# DVA HW4 Q2

Do not distribute or publish this code

Do not change the **#export** statements or add and other code or comments above them. They are needed for grading.

In [2]:
#export
'''
*** Imports ***
    DO NOT EDIT or add anything to this section
'''
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random

## Update 'Utility' class

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [ ]:
#export
class Utility(object):

    def entropy(self, class_y):
        entropy = 0
        ### Implement your code here
        #############################################
        if len(class_y) == 0:
            return 0
        # Convert labels to integers to handle strings "0" or "1"
        class_y = [int(x) for x in class_y]
        num_ones = sum(class_y)
        num_zeros = len(class_y) - num_ones
        if num_ones == 0 or num_zeros == 0:
            return 0
        p1 = num_ones / len(class_y)
        p0 = num_zeros / len(class_y)
        entropy = -p1 * log(p1, 2) - p0 * log(p0, 2)
        #############################################
        return entropy

    def partition_classes(self, X, y, split_attribute, split_val):
        X_left = []
        X_right = []
        y_left = []
        y_right = []
        ### Implement your code here
        #############################################
        for i in range(len(X)):
            if X[i][split_attribute] <= split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])
        #############################################
        return (X_left, X_right, y_left, y_right)

    def information_gain(self, previous_y, current_y):
        info_gain = 0
        ### Implement your code here
        #############################################
        prev_entropy = self.entropy(previous_y)
        left_entropy = self.entropy(current_y[0])
        right_entropy = self.entropy(current_y[1])
        left_weight = len(current_y[0]) / len(previous_y)
        right_weight = len(current_y[1]) / len(previous_y)
        info_gain = prev_entropy - (left_weight * left_entropy + right_weight * right_entropy)
        #############################################
        return info_gain

    def best_split(self, X, y):
        split_attribute = 0
        split_val = 0
        X_left, X_right, y_left, y_right = [], [], [], []
        ### Implement your code here
        #############################################
        max_info_gain = -float('inf')
        num_features = int(len(X[0]) ** 0.5)  # Random subset of features
        feature_indices = random.sample(range(len(X[0])), num_features)
        
        for attr in feature_indices:
            values = sorted(set([row[attr] for row in X]))
            if len(values) < 2:  # Skip if not enough distinct values
                continue
            split_candidates = [(values[i] + values[i+1]) / 2 for i in range(len(values)-1)]
            
            for val in split_candidates:
                X_l, X_r, y_l, y_r = self.partition_classes(X, y, attr, val)
                if len(y_l) == 0 or len(y_r) == 0:
                    continue
                curr_y = [y_l, y_r]
                gain = self.information_gain(y, curr_y)
                if gain > max_info_gain:
                    max_info_gain = gain
                    split_attribute = attr
                    split_val = val
                    X_left, X_right, y_left, y_right = X_l, X_r, y_l, y_r
        
        if max_info_gain <= 0:  # No valid split found
            return {
                'split_attribute': 0,
                'split_val': 0,
                'X_left': [],
                'X_right': [],
                'y_left': [],
                'y_right': [],
                'info_gain': 0
            }
        
        return {
            'split_attribute': split_attribute,
            'split_val': split_val,
            'X_left': X_left,
            'X_right': X_right,
            'y_left': y_left,
            'y_right': y_right,
            'info_gain': max_info_gain
        }
        #############################################

## Update the 'DecisionTree' and 'RandomForest' classes

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [ ]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        self.tree = {}
        self.max_depth = max_depth

    def learn(self, X, y, par_node = {}, depth=0):
        ### Implement your code here
        #############################################
        # Stop if pure, max depth reached, or too few samples
        y_int = [int(label) for label in y]  # Convert for set comparison
        if len(set(y_int)) == 1 or depth >= self.max_depth or len(X) < 2:
            majority = max(set(y), key=y.count)  # Keep original type
            self.tree = {'leaf': True, 'class': majority}
            return
        
        util = Utility()
        split_info = util.best_split(X, y)
        
        if split_info['info_gain'] <= 0 or not split_info['X_left']:
            majority = max(set(y), key=y.count)
            self.tree = {'leaf': True, 'class': majority}
            return
        
        self.tree = {
            'leaf': False,
            'split_attribute': split_info['split_attribute'],
            'split_val': split_info['split_val'],
            'left': DecisionTree(self.max_depth),
            'right': DecisionTree(self.max_depth)
        }
        
        self.tree['left'].learn(split_info['X_left'], split_info['y_left'], depth=depth+1)
        self.tree['right'].learn(split_info['X_right'], split_info['y_right'], depth=depth+1)
        #############################################

    def classify(self, record):
        ### Implement your code here
        #############################################
        node = self.tree
        while not node['leaf']:
            attr = node['split_attribute']
            val = node['split_val']
            if record[attr] <= val:
                node = node['left'].tree
            else:
                node = node['right'].tree
        return node['class']  # Return original type from training
        #############################################

In [ ]:
#export
class RandomForest(object):
    num_trees = 0
    decision_trees = []
    bootstraps_datasets = []
    bootstraps_labels = []

    def __init__(self, num_trees):
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []

    def _bootstrapping(self, XX, n):
        sample = []
        labels = []
        ### Implement your code here
        #############################################
        indices = [random.randint(0, n-1) for _ in range(n)]
        for idx in indices:
            sample.append(XX[idx][:-1])
            labels.append(XX[idx][-1])  # Preserve original label type
        #############################################
        return (sample, labels)

    def bootstrapping(self, XX):
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        ### Implement your code here
        #############################################
        for i in range(self.num_trees):
            self.decision_trees[i].learn(self.bootstraps_datasets[i], self.bootstraps_labels[i])
        #############################################

    def voting(self, X):
        y = []
        for record in X:
            votes = []
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    vote = OOB_tree.classify(record)
                    votes.append(vote)
            if len(votes) == 0:
                ### Implement your code here
                #############################################
                all_votes = [tree.classify(record) for tree in self.decision_trees]
                majority_vote = max(set(all_votes), key=all_votes.count)
                y.append(majority_vote)
                #############################################
            else:
                counts = {}
                for vote in votes:
                    counts[vote] = counts.get(vote, 0) + 1
                majority_vote = max(counts, key=counts.get)
                y.append(majority_vote)
        return np.array(y)

    def user(self):
        ### Implement your code here
        #############################################
        return 'your GT ID' 
        #############################################

In [ ]:
#export

# TODO: Determine the forest size according to your implementation.
# This function will be used by the autograder to set your forest size during testing
# VERY IMPORTANT: Minimum forest_size should be 10
def get_forest_size():
    forest_size = 25
    return forest_size

# TODO: Determine random seed to set for reproducibility
# This function will be used by the autograder to set the random seed to obtain the same results you achieve locally
def get_random_seed():
    random_seed = 42
    return random_seed

## Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run. Similar steps will be used to build and run your code in Gradescope. Any additional testing of functions can be done in the cells below the `run()` cell, as these will not be parsed by the autograder.

In [6]:
def run():
    np.random.seed(get_random_seed())
    # start time
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 31)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("Wisconsin_breast_prognostic.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [8]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()